In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

In [ ]:
games = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/games.csv")
player_play = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/player_play.csv")
players = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/players.csv")
plays = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/plays.csv")
tracking_week_1 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv")
#tracking_week_2 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv")
#tracking_week_3 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv")
#tracking_week_4 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv")
#tracking_week_5 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv")
#tracking_week_6 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv")
#tracking_week_7 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv")
#tracking_week_8 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv")
#tracking_week_9 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv")

In [ ]:
games.head()

In [ ]:
print(player_play.shape)
player_play.head()

In [ ]:
print(players.shape)
players.head()

In [ ]:
print(plays.shape)
plays.head()

In [ ]:
plays.sort_values(by=["gameId", "playId"])

In [ ]:
print(tracking_week_1.shape)
tracking_week_1.head()

In [ ]:
tracking_week_1.gameId.unique()

In [ ]:
tracking_week_1.playDirection.unique()

In [ ]:
tracking_week_1[tracking_week_1.gameId == 2022091200].playId.unique()

In [ ]:
tracking_week_1.frameType.unique()

In [ ]:
play1 = tracking_week_1[(tracking_week_1.gameId == 2022091200) & (tracking_week_1.playId == 64)]

In [ ]:
print(tracking_week_1.x.aggregate(["min", "max"]))
print(tracking_week_1.y.aggregate(["min", "max"]))

In [ ]:
before_snap = play1[play1.frameType == "BEFORE_SNAP"]
snap = play1[play1.frameType == "SNAP"]
after_snap = play1[play1.frameType == "AFTER_SNAP"]

In [ ]:
players = before_snap.displayName.unique()
players

In [ ]:
player0 = before_snap[before_snap.displayName == players[0]]
player0

In [ ]:
player0.loc[:, ["x", "y"]].isna().sum()

In [ ]:
player0.time.aggregate(["min", "max"])

In [ ]:
player.displayName.reset_index(drop=True)

In [ ]:
n = 0
frames = []
for frameType in [before_snap, snap, after_snap]:
    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    for i in range(len(frameType.displayName.unique())):
        player = frameType[frameType.displayName == players[i]].reset_index(drop=True)
        if player.displayName[0] == "football":
            plt.scatter(player.x, player.y, c="red")
            plt.text(list(player.x)[-1], list(player.y)[-1], "FB")
        else:
            plt.scatter(player.x, player.y, c=player.frameId)
            #plt.text(list(player.x)[-1], list(player.y)[-1], list(player.jerseyNumber)[-1])
    
    title = ""
    match n:
        case 0: 
            title = "before snap"
        case 1: 
            title = "snap"
        case 2: 
            title = "after snap"
    plt.title(title)
    
    plt.show()
    n+=1

In [ ]:
play1.frameId.unique()

In [ ]:
frames = []
i = 0
for frame in play1.frameId.unique():
    #frames.append([plt.figure()])
    play_at_frame = play1[play1.frameId == frame].sort_values(by="jerseyNumber").reset_index(drop=True)
    players = play_at_frame[:-1]
    ball = play_at_frame[-1:] 

    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    plt.scatter(players.x, players.y, c=range(len(players)))
    plt.scatter(ball.x, ball.y, c="red", marker="*") 
    plt.title(play_at_frame.frameType[0])
    
    #frames.append(plt.gcf())
    plt.savefig(f"img{i}.png")
    plt.close()
    i += 1

In [ ]:
play_length = (pd.to_datetime(play1.time.max()) - pd.to_datetime(play1.time.min())).total_seconds()
num_frames = len(play1.frameId.unique())
print(f"Frame rate: {num_frames / play_length}")

In [ ]:
os.system("ffmpeg -r 10.06 -i img%01d.png -vcodec mpeg4 -y movie.mp4")